# Implementation Lemmatizer
- Muss: deutscher Lemmatizer bauen.
- Optional: Kann mit Rechtschreibefehler zu einem gewissen Grad umgehen.


Daten: http://wortschatz.uni-leipzig.de/de/

Lizenz: Creative-Commons-Lizenz CC BY-NC

**Sie wünschen zitiert zu werden (In der BDA Dokumentation)**

D. Goldhahn, T. Eckart & U. Quasthoff: Building Large Monolingual Dictionaries at the Leipzig Corpora Collection: From 100 to 200 Languages.
In: Proceedings of the 8th International Language Ressources and Evaluation (LREC'12), 2012

In [3]:
%config IPCompleter.greedy=True

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

### Wörterbuch

In [4]:
import pandas as pd

filename = './de_ch_vocabulary.csv'

word = 'word'
base = 'base'
names = [word, base]

voc = pd.read_csv(filename, sep=',', encoding='ansi', comment='#', names=names)
voc.head()

,word,base
0,ist,sein
1,ein,einen
2,eine,einen
3,eine,ein
4,hat,haben


In [5]:
len(voc)

944382

### Text für Lemmatisation

In [6]:
text = 'In der Philosophie (altgriechisch φιλοσοφία philosophía, latinisiert philosophia, wörtlich „Liebe zur Weisheit“) wird versucht, die Welt und die menschliche Existenz zu ergründen, zu deuten und zu verstehen. Von anderen Wissenschaftsdisziplinen unterscheidet sich die Philosophie dadurch, dass sie sich oft nicht auf ein spezielles Gebiet oder eine bestimmte Methodologie begrenzt, sondern durch die Art ihrer Fragestellungen und ihre besondere Herangehensweise an ihre vielfältigen Gegenstandsbereiche charakterisiert ist. In diesem Artikel wird die westliche (auch: abendländische) Philosophie, die im 6. Jahrhundert v. Chr. im antiken Griechenland entstand, behandelt. Nicht behandelt werden hier die mit der abendländischen Philosophie in einem mannigfaltigen Zusammenhang stehenden Traditionen der jüdischen und der islamischen Philosophie sowie die ursprünglich von ihr unabhängigen Traditionen der afrikanischen und der östlichen Philosophie. In der antiken Philosophie entfaltete sich das systematische und wissenschaftlich orientierte Denken. Im Lauf der Jahrhunderte differenzierten sich die unterschiedlichen Methoden und Disziplinen der Welterschließung und der Wissenschaften direkt oder mittelbar aus der Philosophie, zum Teil auch in Abgrenzung zu irrationalen oder religiösen Weltbildern oder Mythen. Kerngebiete der Philosophie sind die Logik (als die Wissenschaft des folgerichtigen Denkens), die Ethik (als die Wissenschaft des rechten Handelns) und die Metaphysik (als die Wissenschaft der ersten Gründe des Seins und der Wirklichkeit). Weitere Grunddisziplinen sind die Erkenntnis- und Wissenschaftstheorie, die sich mit den Möglichkeiten des Erkenntnisgewinns im Allgemeinen bzw. speziell mit den Erkenntnisweisen der unterschiedlichen Einzelwissenschaften beschäftigen.'
text

'In der Philosophie (altgriechisch φιλοσοφία philosophía, latinisiert philosophia, wörtlich „Liebe zur Weisheit“) wird versucht, die Welt und die menschliche Existenz zu ergründen, zu deuten und zu verstehen. Von anderen Wissenschaftsdisziplinen unterscheidet sich die Philosophie dadurch, dass sie sich oft nicht auf ein spezielles Gebiet oder eine bestimmte Methodologie begrenzt, sondern durch die Art ihrer Fragestellungen und ihre besondere Herangehensweise an ihre vielfältigen Gegenstandsbereiche charakterisiert ist. In diesem Artikel wird die westliche (auch: abendländische) Philosophie, die im 6. Jahrhundert v. Chr. im antiken Griechenland entstand, behandelt. Nicht behandelt werden hier die mit der abendländischen Philosophie in einem mannigfaltigen Zusammenhang stehenden Traditionen der jüdischen und der islamischen Philosophie sowie die ursprünglich von ihr unabhängigen Traditionen der afrikanischen und der östlichen Philosophie. In der antiken Philosophie entfaltete sich das sy

In [7]:
from nltk import tokenize

def lemm_word(w):
    base_word = voc.loc[voc[word] == w.lower(), base]
    return w.lower() if base_word.empty else base_word.iloc[0]

def lemm_sentence(s):
    return str.join(' ', [lemm_word(w) for w in tokenize.word_tokenize(s)])

In [8]:
lemm_sentence(text)

'in der philosophie ( altgriechisch φιλοσοφία philosophía , latinisieren philosophia , wörtlich „ lieb zur weisheit “ ) werden versuchen , die welt und die menschlich existenz zu ergründen , zu deuten und zu verstehen . von andere wissenschaftsdisziplinen unterscheiden sich die philosophie dadurch , dass sie sich oft nicht auf einen speziell gebieten oder einen bestimmt methodologie begrenzen , sondern durch die art ihrer fragestellungen und ihre besondere herangehensweise an ihre vielfältig gegenstandsbereiche charakterisieren sein . in diesem artikel werden die westlich ( auch : abendländisch ) philosophie , die im 6 . jahrhundert v. chr . im antik griechenland entstehen , behandeln . nicht behandeln werden hier die mit der abendländisch philosophie in einem mannigfaltig zusammenhang stehend traditionen der jüdisch und der islamisch philosophie sowie die ursprünglich von ihr unabhängig traditionen der afrikanisch und der östlich philosophie . in der antik philosophie entfaltet sich d

### Rechtschreibefehler Lösungsversuch

In [66]:
import itertools
import Levenshtein

def similarity(w1, w2):
    return Levenshtein.jaro(w1.lower(), w2.lower())

voc['sim'] = voc[word].apply(lambda w: similarity(str(w), 'passwwort'))

In [67]:
voc.iloc[voc['sim'].idxmax()]

word    Passworts
base     Passwort
sim      0.925926
Name: 194339, dtype: object

#### Funktioniert recht gut

In [73]:
# ticket 2019011563000030
def correct(subject):
    voc['sim'] = voc[word].apply(lambda w: similarity(str(w), subject))
    return voc.iloc[voc['sim'].idxmax()][base]

[correct(w) for w in tokenize.word_tokenize('benuttzernamen und passwwort verggessen')]

['Benutzername', 'un', 'Passwort', 'vergessen']